#Preprocessing

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def denoise_image_nlm(image, h=10, hForColorComponents=10, templateWindowSize=7, searchWindowSize=21):
    # Check if the image is loaded correctly and if it has 3 or 4 channels
    if image is None:
        raise ValueError("Image not loaded properly.")
    if image.shape[2] not in [3, 4]:
        raise ValueError("Input image must have 3 (CV_8UC3) or 4 (CV_8UC4) channels.")

    # Apply Non-Local Means Denoising
    denoised_image = cv2.fastNlMeansDenoisingColored(image, None, h, hForColorComponents, templateWindowSize, searchWindowSize)
    return denoised_image

def sharpen_image_high_freq_emphasis(image, kernel=None):
    # Convert to float32 for precision
    image_float = np.float32(image) / 255.0

    # Create a high-pass filter kernel if not provided
    if kernel is None:
        kernel = np.array([[-1, -1, -1],
                           [-1,  10, -1],
                           [-1, -1, -1]])

    # Apply the high-pass filter
    high_freq_image = cv2.filter2D(image_float, -1, kernel)

    # Clip values to the valid range [0, 1] and convert back to uint8
    high_freq_image = np.clip(high_freq_image, 0, 1)
    sharpened_image = (high_freq_image * 255).astype(np.uint8)

    return sharpened_image

def denoise_and_sharpen(image_path, denoise_params={}, sharpen_kernel=None):
    # Load the image
    image = cv2.imread("img.png")

    if image is None:
        raise ValueError("The input image could not be loaded. Please check the path.")

    # Step 1: Denoise
    denoised_image = denoise_image_nlm(image, **denoise_params)

    # Step 2: Sharpen
    sharpened_image = sharpen_image_high_freq_emphasis(denoised_image, sharpen_kernel)

    return denoised_image, sharpened_image

def show_images(original, denoised, sharpened):
    # Convert images from BGR to RGB
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    denoised_rgb = cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB)
    sharpened_rgb = cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB)

    # Plot the images




    plt.imshow(denoised_rgb)

    plt.show()

    plt.imshow(sharpened_rgb)


    plt.show()

# Example usage
input_image_path = 'img.png'

# Adjust these parameters to increase or decrease the effect
denoise_params = {
    'h': 12,  # Increase for more aggressive denoising
    'hForColorComponents': 12,  # Increase for more aggressive denoising of color components
    'templateWindowSize': 15,  # Typically, keep this between 7 and 21
    'searchWindowSize': 30  # Typically, keep this between 21 and 35
}

# Adjust this kernel for sharpening effect
sharpen_kernel = np.array([[-1, -1, -1],
                           [-1, 9.8, -1],  # Increase the central value for stronger sharpening
                           [-1, -1, -1]])

original_image = cv2.imread(input_image_path)

denoised_image, sharpened_image = denoise_and_sharpen(input_image_path, denoise_params, sharpen_kernel)

# Show the images
show_images(original_image, denoised_image, sharpened_image)


#Segmentation

In [ ]:
import cv2
import numpy as np
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral, create_pairwise_gaussian
import matplotlib.pyplot as plt
from skimage.io import imread

def load_image_and_initial_mask(image_path, mask_path=None):
    """
    Load input aerial image and optional initial segmentation mask.
    If mask is not provided, dummy softmax probabilities will be used.
    """
    image = imread(image_path)
    if mask_path:
        mask = cv2.imread(mask_path, 0)  # Load mask as grayscale
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
        # Normalize and make binary softmax-like probabilities
        softmax = np.zeros((2, image.shape[0], image.shape[1]), dtype='float32')
        softmax[1, :, :] = mask / 255.0
        softmax[0, :, :] = 1.0 - softmax[1, :, :]
    else:
        # Dummy softmax: 50% chance for both classes
        softmax = np.full((2, image.shape[0], image.shape[1]), 0.5, dtype='float32')
    return image, softmax

def apply_dense_crf(image, softmax,
                    sxy_gaussian=(3, 3), compat_gaussian=3,
                    sxy_bilateral=(80, 80), srgb_bilateral=(13, 13, 13), compat_bilateral=10,
                    iterations=10):
    """
    Apply Dense CRF to refine the segmentation using the image and initial softmax.
    """
    h, w = image.shape[:2]
    d = dcrf.DenseCRF2D(w, h, 2)  # Two labels: vehicle, background

    # Unary potentials (negative log-probability)
    unary = unary_from_softmax(softmax)
    d.setUnaryEnergy(unary)

    # Pairwise Gaussian (smoothness kernel)
    gaussian_feats = create_pairwise_gaussian(sxy=sxy_gaussian, shape=image.shape[:2])
    d.addPairwiseEnergy(gaussian_feats, compat=compat_gaussian)

    # Pairwise Bilateral (appearance kernel)
    bilateral_feats = create_pairwise_bilateral(sxy=sxy_bilateral, srgb=srgb_bilateral,
                                                img=image, chdim=2)
    d.addPairwiseEnergy(bilateral_feats, compat=compat_bilateral)

    # Inference (MAP)
    Q = d.inference(iterations)
    return np.argmax(Q, axis=0).reshape((h, w))

def visualize_result(original_img, initial_mask, crf_result):
    """
    Visualize the original image, initial mask, and CRF refined output.
    """
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    axs[0].imshow(original_img)
    axs[0].set_title("Original Image")
    axs[1].imshow(initial_mask[1], cmap='gray')
    axs[1].set_title("Initial Segmentation (Softmax)")
    axs[2].imshow(crf_result, cmap='gray')
    axs[2].set_title("CRF Refined Segmentation")
    for ax in axs:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# ===================== RUNNING SECTION =====================

if __name__ == "__main__":
    # Provide image and optional mask (initial segmentation)
    image_path = 'aerial_image.jpg'         # Replace with your aerial image path
    mask_path = 'initial_mask.png'          # Optional: replace with your initial segmentation mask

    # Load input
    image, softmax_probs = load_image_and_initial_mask(image_path, mask_path)

    # Apply CRF
    crf_result = apply_dense_crf(image, softmax_probs)

    # Visualize
    visualize_result(image, softmax_probs, crf_result)
